In [1]:
# UPDATE TO MATCH PERSONAL DIRECTORY STRUCTURE
DIR = '/Users/anvitirunagari/Desktop/vanity_sizing_project/rtr_review_uncleaned.pkl'

In [2]:
import pandas as pd
import pickle

In [3]:
with open('rtr_reviews_uncleaned.pkl', 'rb') as f:
    data = pickle.load(f)

customers = data[['user_id', 'weight', 'body type', 'height', 'size', 'age','band_size', 'cup_size']]

customers = customers.drop_duplicates(subset='user_id')

customers_complete = customers.dropna()

In [4]:
customers_complete

,user_id,weight,body type,height,size,age,band_size,cup_size
0,420272,137.0,hourglass,68,14,28,34,d
1,273551,132.0,straight & narrow,66,12,36,34,b
3,909926,135.0,pear,65,8,34,34,c
4,151944,145.0,athletic,69,12,27,34,b
5,734848,138.0,athletic,68,8,45,32,b
...,...,...,...,...,...,...,...,...
192339,597756,220.0,full bust,64,35,32,38,dd
192343,838239,150.0,hourglass,60,24,55,36,c
192347,442241,127.0,full bust,65,8,33,34,d
192353,961120,165.0,pear,66,16,31,36,c


In [6]:
customers_complete.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77382 entries, 0 to 192354
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   user_id    77382 non-null  int64   
 1   weight     77382 non-null  float64 
 2   body type  77382 non-null  category
 3   height     77382 non-null  Int64   
 4   size       77382 non-null  category
 5   age        77382 non-null  Int64   
 6   band_size  77382 non-null  category
 7   cup_size   77382 non-null  category
dtypes: Int64(2), category(4), float64(1), int64(1)
memory usage: 3.4 MB


### K-Prototypes

#### Setting up the model

In [8]:
import numpy as np
from kmodes.kprototypes import KPrototypes
import matplotlib.pyplot as plt

In [12]:
numerical_cols = ["weight", "height", "age"]
categorical_cols = ["body type", "size", "band_size", "cup_size"]
categorical = [2,4,6,7]

In [9]:
stack = np.column_stack((customers_complete[numerical_cols].values, customers_complete[categorical_cols].values))

In [18]:
def k_prototypes(data, categorical, n_clusters):
    kproto = KPrototypes(n_clusters = n_clusters, init='Huang', n_init=1, verbose=0)
    clusters = kproto.fit_predict(data, categorical=categorical)
    return clusters

#### Gap Statistic - used to calculate the best number of clusters

In [14]:
def gap_statistic(data, categorical, k_max):
    gaps = np.zeros((k_max,))
    resultsdf = pd.DataFrame({'clusterCount':[], 'gap':[]})
    for k in range(1, k_max+1):
        kproto = KPrototypes(n_clusters=k, init='Huang', n_init=1, verbose=0)
        kproto.fit_predict(data, categorical = categorical)
        
        refDisps = np.zeros(10)
        for i in range (10):
            randomReference = np.random.random_sample(size=data.shape)
            kproto_ref = KPrototypes(n_clusters=k, init='Huang', n_init=1, verbose=0)
            kproto_ref.fit_predict(randomReference, categorical = categorical)
            refDisp = kproto_ref.cost_
            refDisps[i] = refDisp
        
        originalDisp = kproto.cost_
        gap = np.log(np.mean(refDisps)) - np.log(originalDisp)
        resultsdf = resultsdf.append({'clusterCount':k, 'gap': gap}, ignore_index=True)
    return resultsdf

In [ ]:
gap_statistic(customers_complete, [2,4,6,7], 5)

/var/folders/vk/qv6c5zr14pbdttxdnjls2jxr0000gn/T/ipykernel_22447/1471794751.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultsdf = resultsdf.append({'clusterCount':k, 'gap': gap}, ignore_index=True)


In [19]:
k_prototypes(customers_complete, [2,4,6,7], 15)

array([4, 7, 0, ..., 4, 5, 6], dtype=uint16)